In [1]:
import argparse
import atexit
import multiprocessing
import sys
import py_trees
import py_trees.common
import time

import py_trees.console as console

In [2]:
def description():
    content = "Demonstrates sequences in action.\n\n"
    content += "A sequence is populated with 2-tick jobs that are allowed to run through to\n"
    content += "completion.\n"

    if py_trees.console.has_colours:
        banner_line = console.green + "*" * 79 + "\n" + console.reset
        s = "\n"
        s += banner_line
        s += console.bold_white + "Sequences".center(79) + "\n" + console.reset
        s += banner_line
        s += "\n"
        s += content
        s += "\n"
        s += banner_line
    else:
        s = content
    return s


def epilog():
    if py_trees.console.has_colours:
        return console.cyan + "And his noodly appendage reached forth to tickle the blessed...\n" + console.reset
    else:
        return None


def command_line_argument_parser():
    parser = argparse.ArgumentParser(description=description(),
                                     epilog=epilog(),
                                     formatter_class=argparse.RawDescriptionHelpFormatter,
                                     )
    parser.add_argument('-r', '--render', action='store_true', help='render dot tree to file')
    return parser


def create_root():
    root = py_trees.composites.Sequence("Sequence")
    for action in ["Action 1", "Action 2", "Action 3"]:
        success_after_two = py_trees.behaviours.Count(name=action,
                                                      fail_until=0,
                                                      running_until=1,
                                                      success_until=10)
        root.add_child(success_after_two)
    return root


##############################################################################
# Main
##############################################################################

def main():
    """
    Entry point for the demo script.
    """
    #args = command_line_argument_parser().parse_args()
    print(description())
    py_trees.logging.level = py_trees.logging.Level.DEBUG

    root = create_root()

    ####################
    # Rendering
    ####################
    #if args.render:
    #py_trees.display.render_dot_tree(root)
    #sys.exit()

    ####################
    # Execute
    ####################
    root.setup_with_descendants()
    for i in range(1, 6):
        try:
            print("\n--------- Tick {0} ---------\n".format(i))
            root.tick_once()
            print("\n")
            print(py_trees.display.unicode_tree(root=root, show_status=True))
            time.sleep(1.0)
        except KeyboardInterrupt:
            break
    print("\n")

In [3]:
a=main()

Demonstrates sequences in action.

A sequence is populated with 2-tick jobs that are allowed to run through to
completion.


--------- Tick 1 ---------

[DEBUG] Sequence             : Sequence.tick()
[DEBUG] Sequence             : Sequence.tick() [!RUNNING->resetting child index]
[DEBUG] Action 1             : Count.tick()
[DEBUG] Action 1             : Count.update()[1: running]


[-] Sequence [*]
    --> Action 1 [*] -- running
    --> Action 2 [-]
    --> Action 3 [-]


--------- Tick 2 ---------

[DEBUG] Sequence             : Sequence.tick()
[DEBUG] Action 1             : Count.tick()
[DEBUG] Action 1             : Count.update()[2: success]
[DEBUG] Action 1             : Count.stop(Status.RUNNING->Status.SUCCESS)
[DEBUG] Action 1             : Count.terminate(Status.RUNNING->Status.SUCCESS)
[DEBUG] Action 2             : Count.tick()
[DEBUG] Action 2             : Count.update()[1: running]


[-] Sequence [*]
    --> Action 1 [o]
    --> Action 2 [*] -- running
    --> Action 3 [